# 分类: 基础学习器

In [20]:
import lightgbm as lgbm
import pandas as pd 
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [21]:
def get_train_val(val_id):
    '''获取训练集和校验集 
    val_id = 0,1,2 #当前校验集编号 '''
    print('val_id=%s'% (val_id))
    df_val = train_list[val_id]
    train_tmp = []

    for i in range(1, train_part_num):
        cur_id = (val_id + i) % 5
        print('cur_id=%s'% (cur_id))
        train_tmp.append(train_list[cur_id])
    
#     只生成一次，节省内存 
    df_train = pd.concat(train_tmp)
    print('df_train.shape=%s  df_val.shape=%s'% (df_train.shape, df_val.shape))
    return df_train, df_val

def generate_summit(predict, df):
    ''' 输出预测后的数据 '''
    testPredict = df.copy()
    testPredict[target] = predict
    testPredict[target] = testPredict[target].astype(np.float32)
#     testPredict = testPredict[[user_id, target]]
#     应该过滤掉负数
#     testPredict[target] = testPredict[target].apply(lambda x: x if x > 0 else 0)
    return testPredict

In [22]:
path_train = '../../data/new/train=all-class/' 
path_test = '../../data/new/test/'

val_path = '../../data/val/classBase/' #保存校验集测试结果 
val_name = '%sval=%s-%s.csv'
summit_path = '../../data/new/test=class/' #保存提交文件，测试集测试结果 
out_name = '%stest-7pay=%s-%s.csv' #生成的结果文件名称
base = 'classBase' #学习器级别 

train_part_num = 5 #训练集拆分个数
target = 'prediction_pay_price'
user_id = 'user_id'

## 指定读取数据类型，节省内存

In [23]:
col_int32 = ['user_id', 'register_time', 'wood_add_value', 'wood_reduce_value',
       'stone_add_value', 'stone_reduce_value', 'ivory_add_value',
       'ivory_reduce_value', 'meat_add_value', 'meat_reduce_value',
       'magic_add_value', 'magic_reduce_value', 'infantry_add_value',
       'infantry_reduce_value', 'cavalry_add_value', 'cavalry_reduce_value',
       'shaman_add_value', 'shaman_reduce_value', 'wound_infantry_add_value',
       'wound_infantry_reduce_value', 'wound_cavalry_add_value',
       'wound_cavalry_reduce_value', 'wound_shaman_add_value',
       'wound_shaman_reduce_value', 'general_acceleration_add_value',
       'general_acceleration_reduce_value', 'building_acceleration_add_value',
       'building_acceleration_reduce_value', 'reaserch_acceleration_add_value',
       'reaserch_acceleration_reduce_value', 'training_acceleration_add_value',
       'training_acceleration_reduce_value', 'treatment_acceleraion_add_value',
       'treatment_acceleration_reduce_value', 'bd_training_hut_level',
       'bd_healing_lodge_level', 'bd_stronghold_level',
       'bd_outpost_portal_level', 'bd_barrack_level',
       'bd_healing_spring_level', 'bd_dolmen_level', 'bd_guest_cavern_level',
       'bd_warehouse_level', 'bd_watchtower_level', 'bd_magic_coin_tree_level',
       'bd_hall_of_war_level', 'bd_market_level', 'bd_hero_gacha_level',
       'bd_hero_strengthen_level', 'bd_hero_pve_level', 'sr_scout_level', 
       'sr_training_speed_level', 'sr_infantry_tier_2_level',
       'sr_cavalry_tier_2_level', 'sr_shaman_tier_2_level',
       'sr_infantry_atk_level', 'sr_cavalry_atk_level', 'sr_shaman_atk_level',
       'sr_infantry_tier_3_level', 'sr_cavalry_tier_3_level',
       'sr_shaman_tier_3_level', 'sr_troop_defense_level',
       'sr_infantry_def_level', 'sr_cavalry_def_level', 'sr_shaman_def_level',
       'sr_infantry_hp_level', 'sr_cavalry_hp_level', 'sr_shaman_hp_level',
       'sr_infantry_tier_4_level', 'sr_cavalry_tier_4_level',
       'sr_shaman_tier_4_level', 'sr_troop_attack_level',
       'sr_construction_speed_level', 'sr_hide_storage_level',
       'sr_troop_consumption_level', 'sr_rss_a_prod_levell',
       'sr_rss_b_prod_level', 'sr_rss_c_prod_level', 'sr_rss_d_prod_level',
       'sr_rss_a_gather_level', 'sr_rss_b_gather_level',
       'sr_rss_c_gather_level', 'sr_rss_d_gather_level', 'sr_troop_load_level',
       'sr_rss_e_gather_level', 'sr_rss_e_prod_level',
       'sr_outpost_durability_level', 'sr_outpost_tier_2_level',
       'sr_healing_space_level', 'sr_gathering_hunter_buff_level',
       'sr_healing_speed_level', 'sr_outpost_tier_3_level',
       'sr_alliance_march_speed_level', 'sr_pvp_march_speed_level',
       'sr_gathering_march_speed_level', 'sr_outpost_tier_4_level',
       'sr_guest_troop_capacity_level', 'sr_march_size_level',
       'sr_rss_help_bonus_level', 'pvp_battle_count', 'pvp_lanch_count',
       'pvp_win_count', 'pve_battle_count', 'pve_lanch_count', 'pve_win_count',
       'pay_count']
col_float32 = ['avg_online_minutes', 'pay_price']

In [24]:
# 指定读取数据类型，节省内存
dtype_test = {}
for col in col_int32:
    dtype_test[col] = np.int32
for col in col_float32:
    dtype_test[col] = np.float32
# dtype_test

In [25]:
dtype_train = dtype_test.copy()
dtype_train[target] = np.int32
# dtype_train

## 读取数据，一次读取多次使用，不同组合 

In [26]:
train_list = []
for i in range(train_part_num):
    cur_id = i + 1
    train_list.append(pd.read_csv('%strain_7pay_45-%s.csv' % (path_train, cur_id), dtype=dtype_train))
train_list[0].head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,342400,105,8615634,7917409,3411059,2177688,1853501,0,10225418,7357320,...,5,0,3,72,72,62,423.333344,0.990000,1,1
1,3031144,124,4568971,4103711,2059099,1153648,693000,0,5841268,3053325,...,12,8,7,43,43,37,38.000000,0.990000,1,0
2,382912,135,2443311,2088353,1440414,1440864,219000,0,6655247,5426899,...,3,2,2,36,36,36,163.000000,0.990000,1,0
3,1489295,129,10763523,10735318,5948087,5325936,3096454,0,16787348,10258627,...,3,0,0,86,86,78,323.000000,15.970000,3,0
4,2233092,100,13496537,11672639,10185337,7662464,3563466,0,27806092,18809536,...,3,1,1,18,18,13,41.500000,20.959999,4,0


## 5折交叉验证

In [39]:
# 根据校验集编号 0，1,2,3,4 ，获取训练集和校验集
df_train, df_val = get_train_val(4)

# 从原始数据中分离输入特征x和输出y
train_y = df_train[target].values
train_X = df_train.drop([target, user_id, 'register_time'], axis = 1)
val_y = df_val[target].values
val_X = df_val.drop([target, user_id, 'register_time'], axis = 1)

val_id=4
cur_id=0
cur_id=1
cur_id=2
cur_id=3
df_train.shape=(33151, 109)  df_val.shape=(8288, 109)


In [40]:
# 数据标准化：分别初始化对特征和目标值的标准化器 
ss_X = StandardScaler()
ss_y = StandardScaler()

# 分别对训练和测试数据的特征以及目标值进行标准化处理
train_X_ss = ss_X.fit_transform(train_X)
val_X_ss = ss_X.transform(val_X)

# 需要转换为一维数组 
train_y_ss_1d = train_y.reshape(train_y.shape[0])
val_y_ss_1d = val_y.reshape(val_y.shape[0])

In [41]:
test = pd.read_csv(path_test + 'test-7pay.csv', dtype=dtype_test)
test_X = test.drop([user_id, 'register_time'], axis = 1)
test_X_ss = ss_X.transform(test_X)
test.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,sr_rss_help_bonus_level,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count
0,14952,94,3838322,2859288,1041000,0,504000,0,6898146,4297996,...,0,4,4,4,4,4,3,27.166668,0.99,1
1,15029,94,2353190,1286349,1000000,0,609000,0,3216753,513123,...,0,2,0,0,5,5,5,19.500000,0.99,1
2,15059,94,6058137,3960541,5040000,2520000,2109000,0,14586512,7910807,...,0,0,0,0,11,11,9,13.833333,5.98,2
3,15064,94,2958155,2350347,1364518,407296,600000,0,5895581,3241016,...,0,2,1,0,2,2,2,13.000000,0.99,1
4,15102,94,1080008,19659,1000000,0,500000,0,2303911,13076,...,0,0,0,0,1,1,1,2.166667,0.99,1


## 模型训练：1. LightGBM
https://blog.csdn.net/owenfy/article/details/79631144
- 直接调用 LightGBM 内嵌的交叉验证（cv），可对连续的 n_estimators 参数进行快速交叉验证
- 而 GridSearchCV 只能对有限个参数进行交叉验证

In [42]:
# %%time
# lgbm_1 = lgbm.LGBMClassifier(silent=False)
# param_dist = {"max_depth": [25,50, 75],
#               "learning_rate" : [0.01,0.05,0.1],
#               "num_leaves": [300,900,1200],
#               "n_estimators": [200]
#              }
# grid_search = GridSearchCV(lgbm_1, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
# grid_search.fit(train_X_ss, train_y_ss_1d)

In [43]:
# grid_search.best_estimator_

In [44]:
# # 模型评估
# predict_train = grid_search.predict(train_X_ss)
# scoreTrain = metrics.roc_auc_score(train_y_ss_1d, predict_train)
# predict_val = grid_search.predict(val_X_ss)
# scoreVal = metrics.roc_auc_score(val_y_ss_1d, predict_val)
# print('The score of model: Train=%s  Val=%s'% (scoreTrain, scoreVal))
# # The score of model: Train=0.9997237263786054  Val=0.7304572760744842 

In [45]:
%%time
d_train = lgbm.Dataset(train_X_ss, label=train_y_ss_1d)
params = {"max_depth": 55, "learning_rate" : 0.05, "num_leaves": 1000,  "n_estimators": 99}
model_2 = lgbm.train(params, d_train)

d:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Wall time: 12.9 s


In [46]:
# 模型评估
predict_train = model_2.predict(train_X_ss)
scoreTrain = metrics.roc_auc_score(train_y_ss_1d, predict_train)
predict_val = model_2.predict(val_X_ss)
scoreVal = metrics.roc_auc_score(val_y_ss_1d, predict_val)
print('The score of model: Train=%s  Val=%s'% (scoreTrain, scoreVal))

# "max_depth": 50, "learning_rate" : 0.05, "num_leaves": 900,  "n_estimators": 200
# "max_depth": 50, "learning_rate" : 0.05, "num_leaves": 900,  "n_estimators": 150
# "max_depth": 50, "learning_rate" : 0.05, "num_leaves": 900,  "n_estimators": 100
# "max_depth": 50, "learning_rate" : 0.05, "num_leaves": 900,  "n_estimators": 50
# "max_depth": 50, "learning_rate" : 0.05, "num_leaves": 900,  "n_estimators": 60
# "max_depth": 55, "learning_rate" : 0.05, "num_leaves": 900,  "n_estimators": 60
# "max_depth": 55, "learning_rate" : 0.05, "num_leaves": 800,  "n_estimators": 60
# "max_depth": 55, "learning_rate" : 0.05, "num_leaves": 850,  "n_estimators": 60
# "max_depth": 55, "learning_rate" : 0.05, "num_leaves": 830,  "n_estimators": 60
# "max_depth": 55, "learning_rate" : 0.05, "num_leaves": 820,  "n_estimators": 60
# "max_depth": 55, "learning_rate" : 0.05, "num_leaves": 820,  "n_estimators": 65
# The score of model: Train=0.9972400950691207  Val=0.851761910494389
# The score of model: Train=0.9995678579093338  Val=0.8409847645263386

The score of model: Train=0.9993456809648116  Val=0.8408876920081907


In [47]:
valPredict = generate_summit(predict_val, df_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'lightgbm'), index=False)
valPredict.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,1096956,107,6174,7305,0,0,0,0,4763,4140,...,0,0,0,0,0,0,3.333333,10.970000,3,0.021747
1,1812886,124,4763308,4338016,2627000,1428000,1035000,0,7969641,5264255,...,4,1,3,19,19,17,44.166668,2.970000,3,0.096469
2,2676206,120,9114494,9539755,5018154,4972967,1626873,127796,11657301,8940071,...,49,41,42,58,58,46,119.833336,22.950001,5,0.644495
3,2186992,136,28674,0,0,0,0,0,0,0,...,0,0,0,0,0,0,133.333328,0.990000,1,0.002724
4,1880998,101,18938978,19173006,11409936,11238572,5020923,747702,28692940,23392620,...,102,70,76,149,148,138,364.666656,26.940001,6,0.851580


In [48]:
model_pred = model_2.predict(test_X)
print('model_pred:', model_pred)

model_pred: [0.96567732 0.69154638 0.98389762 ... 0.78802859 0.90338372 0.74849915]


In [49]:
#输出预测后的数据
testPredict = generate_summit(model_pred, test)
testPredict.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,14952,94,3838322,2859288,1041000,0,504000,0,6898146,4297996,...,4,4,4,4,4,3,27.166668,0.99,1,0.965677
1,15029,94,2353190,1286349,1000000,0,609000,0,3216753,513123,...,2,0,0,5,5,5,19.500000,0.99,1,0.691546
2,15059,94,6058137,3960541,5040000,2520000,2109000,0,14586512,7910807,...,0,0,0,11,11,9,13.833333,5.98,2,0.983898
3,15064,94,2958155,2350347,1364518,407296,600000,0,5895581,3241016,...,2,1,0,2,2,2,13.000000,0.99,1,0.878806
4,15102,94,1080008,19659,1000000,0,500000,0,2303911,13076,...,0,0,0,1,1,1,2.166667,0.99,1,1.006986


In [50]:
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'lightgbm'), index=False)
testPredict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19549 entries, 0 to 19548
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float32(3), int32(106)
memory usage: 8.1 MB
